In [5]:
from collections import defaultdict
from datetime import datetime
import torch
import pandas as pd

In [6]:
ml_1m_path = '../../../../datasets/ml-1m'

In [7]:
file_path = ml_1m_path + '/movies.dat'
# file_path = Path(file_path)
data_movies = pd.read_csv(file_path, encoding='latin-1', sep='::', engine='python',
                   names=['MovieID', 'Title', 'Genres'], index_col='MovieID')
genres = data_movies['Genres'].str.get_dummies('|')
data_movies= pd.concat([data_movies, genres], axis=1)
# [row, columns], axis = 1 is column
data_movies.drop('Genres', axis=1, inplace=True)
data_movies

,Title,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
MovieID,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Jumanji (1995),0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,Grumpier Old Men (1995),0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
4,Waiting to Exhale (1995),0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
5,Father of the Bride Part II (1995),0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3948,Meet the Parents (2000),0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3949,Requiem for a Dream (2000),0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3950,Tigerland (2000),0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [8]:
file_path = ml_1m_path + '/users.dat'
data_users = pd.read_csv(file_path, sep='::', engine='python',
                   names=['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code'], index_col='UserID')

# Convert Gender to binary indicator variable
data_users['Gender'] = (data_users['Gender'] == 'M').astype(int)
occupation_map = {
    0: 'other',
    1: 'academic/educator',
    2: 'artist',
    3: 'clerical/admin',
    4: 'college/grad student',
    5: 'customer service',
    6: 'doctor/health care',
    7: 'executive/managerial',
    8: 'farmer',
    9: 'homemaker',
    10: 'K-12 student',
    11: 'lawyer',
    12: 'programmer',
    13: 'retired',
    14: 'sales/marketing',
    15: 'scientist',
    16: 'self-employed',
    17: 'technician/engineer',
    18: 'tradesman/craftsman',
    19: 'unemployed',
    20: 'writer'
}
# data['Occupation'] = data['Occupation'].map(occupation_map)
data_users

,Gender,Age,Occupation,Zip-code
UserID,,,,
1,0,1,10,48067
2,1,56,16,70072
3,1,25,15,55117
4,1,45,7,02460
5,1,25,20,55455
...,...,...,...,...
6036,0,25,15,32603
6037,0,45,1,76006
6038,0,56,1,14706


In [9]:
file_path = ml_1m_path + '/ratings.dat'

data_ratings = pd.read_csv(file_path, sep='::', engine='python',
                   names=['UserID', 'MovieID', 'Rating', 'Timestamp'], index_col='UserID')


In [10]:
data_ratings

,MovieID,Rating,Timestamp
UserID,,,
1,1193,5,978300760
1,661,3,978302109
1,914,3,978301968
1,3408,4,978300275
1,2355,5,978824291
...,...,...,...
6040,1091,1,956716541
6040,1094,5,956704887
6040,562,5,956704746


## SASRec handling
Only use implicit feedback in the sequence of items

In [11]:
data_ratings

,MovieID,Rating,Timestamp
UserID,,,
1,1193,5,978300760
1,661,3,978302109
1,914,3,978301968
1,3408,4,978300275
1,2355,5,978824291
...,...,...,...
6040,1091,1,956716541
6040,1094,5,956704887
6040,562,5,956704746


In [81]:
# use timestamps to determine the sequence order of actions.
actions = data_ratings
actions = actions.groupby('MovieID').filter(lambda  x: len(x) >= 5)
actions = actions.groupby('UserID').filter(lambda  x: len(x) >= 5)
actions = actions.groupby('UserID', group_keys=False).apply(lambda  x: x.sort_values('Timestamp'))
# We discard users and items with fewer than 5 related actions.
actions

,MovieID,Rating,Timestamp
UserID,,,
1,3186,4,978300019
1,1721,4,978300055
1,1022,5,978300055
1,1270,5,978300055
1,2340,3,978300103
...,...,...,...
6040,2917,4,997454429
6040,1784,3,997454464
6040,1921,4,997454464


In [82]:
usermap = dict()
usernum = 0
itemmap = dict()
itemnum = 0
# reorder the userid and itemid (keep the same step with original SASRec code)
for _id, row in actions.iterrows():
    if _id in usermap:
        userid = usermap[_id]
    else:
        usernum += 1
        userid = usernum
        usermap[_id] = userid

    if row.MovieID in itemmap:
        itemid = itemmap[row.MovieID]
    else:
        itemnum += 1
        itemid = itemnum
        itemmap[row.MovieID] = itemid

In [83]:
usermap.__len__(), itemmap.__len__()

(6040, 3416)

In [84]:
actions['MovieID'] = actions['MovieID'].map(itemmap)
actions.index = actions.index.map(usermap)
# data['Occupation'] = data['Occupation'].map(occupation_map)

In [15]:
actions

,MovieID,Rating,Timestamp
UserID,,,
1,1,4,978300019
1,2,4,978300055
1,3,5,978300055
1,4,5,978300055
1,5,3,978300103
...,...,...,...
6040,1249,4,997454429
6040,88,3,997454464
6040,371,4,997454464


In [86]:
sas_data = actions.drop('Rating', axis=1, inplace=False).drop('Timestamp', axis=1, inplace=False)

In [18]:
with open('ml-1m.txt', 'w') as f:
    for _id in set(sas_data.index):
        for movie_id in sas_data.loc[_id].MovieID:
            f.write('%d %d\n' % (_id, movie_id))

In [90]:
day = 86400
usernum = actions.index.max()
split_percent = 0.9
split_index = int(len(actions.loc[3]) * split_percent)
train_seq = actions.loc[3].iloc[:split_index]
target_seq = actions.loc[3].iloc[split_index:]
target_seq.Timestamp.max() - train_seq.Timestamp.max()
# actions.loc[1].Timestamp.max() -

188

In [91]:
userMaxTime = actions.groupby('UserID').Timestamp.max()
userActLength = actions.groupby('UserID').size()
split_index = (userActLength * split_percent).astype(int)
userMaxTime, split_index

(UserID
 1       978824351
 2       978300174
 3       978298504
 4       978294282
 5       978246585
           ...    
 6036    956755196
 6037    956801840
 6038    956717204
 6039    956758029
 6040    998315055
 Name: Timestamp, Length: 6040, dtype: int64,
 UserID
 1        47
 2       116
 3        45
 4        18
 5       178
        ... 
 6036    799
 6037    181
 6038     18
 6039    110
 6040    306
 Length: 6040, dtype: int32)

In [80]:
window_size = []
for i in range(1 ,actions.index.max() + 1):
    cur_seq = actions.loc[i]
    split = split_index.loc[i]
    train_seq = cur_seq.iloc[:split]
    target_seq = cur_seq.iloc[split:]
    window_size.append(target_seq.iloc[-1].Timestamp - train_seq.iloc[-1].Timestamp)

seq_avg_length= userActLength.mean()
seq_avg_length # , window_size

(165.49850993377484,
 [156,
  261,
  338,
  22,
  722,
  1110,
  24,
  15650,
  363,
  1470652,
  5279,
  57,
  160,
  314,
  149252,
  124,
  318,
  1560,
  15127680,
  112,
  117,
  12994916,
  15241369,
  8022472,
  253412,
  113844,
  324,
  858897,
  351,
  129,
  301,
  2645,
  3199,
  293,
  107557,
  46843182,
  295,
  1181,
  120,
  209,
  1551,
  714,
  128,
  1606,
  1033,
  70,
  92,
  342578,
  838908,
  120,
  122,
  648,
  6620,
  102,
  89,
  230,
  282,
  943,
  860,
  77,
  199,
  1753,
  2210449,
  215,
  46524175,
  480,
  579,
  305,
  213,
  129,
  139,
  251,
  3053711,
  406,
  897421,
  344,
  161,
  288,
  259,
  15991573,
  414,
  1041313,
  322,
  356,
  327,
  4505964,
  73585,
  159,
  7046,
  89185,
  127,
  522861,
  1078,
  0,
  1133,
  1186,
  22094588,
  183,
  697,
  437,
  478,
  4003124,
  367,
  81,
  74,
  98,
  249,
  404,
  1021,
  459,
  337,
  73,
  491,
  2867558,
  42,
  810,
  1138434,
  693,
  702569,
  183,
  244,
  131,
  1630409,
  183

In [25]:
def data_partition_window(fname):
    usernum = 0
    itemnum = 0
    User = defaultdict(list)
    split_percent = 0.9
    user_train = {}
    user_valid = {}
    user_test = {}
    # assume user/item index starting from 1
    f = open('%s.txt' % fname, 'r')
    # read from each line
    for line in f:
        u, i = line.rstrip().split(' ')
        u = int(u)
        i = int(i)
        usernum = max(u, usernum)
        itemnum = max(i, itemnum)
        User[u].append(i)
    # read from each user
    count = 1
    for user in User:
        nfeedback = len(User[user])
        if nfeedback < 3:
            user_train[user] = User[user]
            user_valid[user] = []
            user_test[user] = []
        else:
            # select the whole training seq
            # user_train[user] = User[user][:-2]
            train_seq = User[user][:-2]
            train_seq_length =len(train_seq)
            split_index = int(train_seq_length * split_percent)
            input_seq = train_seq[:split_index]
            target_seq = train_seq[split_index:]
            for target in target_seq:
                user_train[count] = input_seq + [target]
                count+=1
            user_valid[user] = []
            user_valid[user].append(User[user][-2])
            user_test[user] = []
            user_test[user].append(User[user][-1])
    return [user_train, user_valid, user_test, usernum, itemnum]

In [27]:
dataset = data_partition_window('ml-1m')

In [28]:
[user_train, user_valid, user_test, usernum, itemnum] = dataset

In [29]:
len(user_train), user_train[101442]

(101442,
 [553,
  341,
  66,
  16,
  1025,
  2760,
  1532,
  1282,
  2509,
  1639,
  322,
  387,
  13,
  3334,
  42,
  2478,
  61,
  772,
  3021,
  2577,
  63,
  14,
  124,
  17,
  1953,
  294,
  390,
  29,
  288,
  801,
  15,
  268,
  2504,
  22,
  543,
  337,
  290,
  2498,
  122,
  282,
  100,
  324,
  1966,
  538,
  466,
  968,
  961,
  116,
  1095,
  279,
  273,
  1270,
  291,
  350,
  465,
  132,
  272,
  458,
  239,
  1165,
  84,
  892,
  1955,
  204,
  348,
  60,
  1529,
  1099,
  565,
  1257,
  58,
  2703,
  391,
  1359,
  55,
  2499,
  1960,
  2560,
  1551,
  214,
  1534,
  1299,
  89,
  54,
  79,
  1306,
  1259,
  1899,
  217,
  99,
  1026,
  806,
  521,
  71,
  200,
  25,
  1816,
  1246,
  20,
  35,
  2890,
  948,
  386,
  741,
  1094,
  634,
  1393,
  118,
  203,
  1092,
  828,
  257,
  259,
  742,
  101,
  577,
  848,
  56,
  236,
  740,
  1192,
  30,
  989,
  1952,
  1260,
  800,
  1428,
  1950,
  1138,
  97,
  1895,
  205,
  184,
  1591,
  1101,
  1302,
  1304,
  2738,


In [37]:
import numpy as np
user = np.random.randint(1, usernum + 1)
user

136

5922

1910

726

# Evaluation metrix modification

In [94]:
from collections import defaultdict

def data_partition(fname):
    usernum = 0
    itemnum = 0
    User = defaultdict(list)
    user_train = {}
    user_valid = {}
    user_test = {}
    # assume user/item index starting from 1
    f = open('%s.txt' % fname, 'r')
    for line in f:
        u, i = line.rstrip().split(' ')
        u = int(u)
        i = int(i)
        usernum = max(u, usernum)
        itemnum = max(i, itemnum)
        User[u].append(i)

    for user in User:
        nfeedback = len(User[user])
        if nfeedback < 3:
            user_train[user] = User[user]
            user_valid[user] = []
            user_test[user] = []
        else:
            user_train[user] = User[user][:-2]
            user_valid[user] = []
            user_valid[user].append(User[user][-2])
            user_test[user] = []
            user_test[user].append(User[user][-1])
    return [user_train, user_valid, user_test, usernum, itemnum]

In [98]:
dataset = data_partition('ml-1m')
dataset[0]

KeyError: 0

In [3]:
import random
import numpy as np
import copy
import torch
import sys

def evaluate_window_valid(model, dataset, args):
    [train, valid, test, usernum, itemnum] = copy.deepcopy(dataset)
    Recall = 0.0
    P90 = 0.0
    # P90 coverage means the smallest item sets that appear in the top 10 lists of at least 90% of the users.
    valid_user = 0.0
    sample_nums = 500
    random_items = random.sample(range(1, itemnum + 1), sample_nums)
    # if usernum > 10000:
    #     # avoid too many training users
    #     # keep at most 10000 users
    #     users = random.sample(range(1, usernum + 1), 10000)
    # else:
    #     # else keep all the users
    #     users = range(1, usernum + 1)
    users = range(1, usernum+1)
    for u in users:
        # make sure the sequence can be validated
        if len(train[u]) < 1 or len(valid[u]) < 1: continue
        seq = np.zeros([args.maxlen], dtype=np.int32)
        idx = args.maxlen - 1
        for i in reversed(train[u]):
            seq[idx] = i
            # fill the sequence from end to beginning
            idx -= 1
            if idx == -1: break
            # select the max len or all of the training data in the sequence
            # limit the length, seq contains the actual training sequence
        rated = set(train[u])
        rated.add(0)
        # all items interacted by the current user
        item_idx = [valid[u][0]]
        # get the index of validated item
        for _ in range(100):
            # negative sampling
            t = np.random.randint(1, itemnum + 1)
            # randomly sample 100 items
            while t in rated: t = np.random.randint(1, itemnum + 1)
            item_idx.append(t)
        predictions = -model.predict(*[np.array(l) for l in [[u], [seq], item_idx]])
        # predicting the recommendation list
        predictions = predictions[0]
        rank = predictions.argsort().argsort()[0].item()
        # the rank of the expected next single item
        valid_user += 1
        if rank < 10:
            Recall += 1
            # P90 coverage
        if valid_user % 100 == 0:
            print('.', end="")
            sys.stdout.flush()
    return Recall / valid_user, P90 / valid_user

In [4]:
import argparse

def str2bool(s):
    if s not in {'false', 'true'}:
        raise ValueError('Not a valid boolean string')
    return s == 'true'

def create_args(args):
    parser = argparse.ArgumentParser()
    parser.add_argument('--dataset', required=True)
    parser.add_argument('--train_dir', required=True)
    parser.add_argument('--batch_size', default=128, type=int)
    parser.add_argument('--lr', default=0.001, type=float)
    parser.add_argument('--maxlen', default=50, type=int)
    parser.add_argument('--hidden_units', default=50, type=int)
    parser.add_argument('--num_blocks', default=2, type=int)
    parser.add_argument('--num_epochs', default=201, type=int)
    parser.add_argument('--num_heads', default=1, type=int)
    parser.add_argument('--dropout_rate', default=0.5, type=float)
    parser.add_argument('--l2_emb', default=0.0, type=float)
    parser.add_argument('--device', default='cpu', type=str)
    parser.add_argument('--inference_only', default=False, type=str2bool)
    parser.add_argument('--state_dict_path', default=None, type=str)
    args = parser.parse_args(args)
    return args

In [5]:
from models.SASRec.model import SASRec
[train, valid, test, usernum, itemnum] = copy.deepcopy(dataset)
model_path = '../../processed/ml-1m_repro2/SASRec.epoch=201.lr=0.001.layer=2.head=1.hidden=50.maxlen=200.pth'
# args.device = 'ml-1m'
# args.train_dir = 'test'
# args.state_dict_path = model_path
# args.inference
args = create_args(['--dataset','ml-1m',
                    '--train_dir', 'test',
                    '--device', 'cuda',
                    '--state_dict_path', model_path,
                    '--inference_only', 'true',
                    '--maxlen', '200'])
# print(args.dataset)
model = SASRec(usernum, itemnum, args).to(args.device) # no ReLU activation in original SASRec implementation?
model.load_state_dict(torch.load(args.state_dict_path, map_location=torch.device(args.device)))


<All keys matched successfully>

In [6]:
# seq = np.zeros([200], dtype=np.int32)
from collections import Counter

def window_eval(model, dataset, args):
    [train, valid, test, usernum, itemnum] = copy.deepcopy(dataset)
    Recall = 0.0
    P90 = 0.0
    coverage_list = []
    # P90 coverage means the smallest item sets that appear in the top 10 lists of at least 90% of the users.
    valid_user = 0.0
    sample_nums = 100
    random_items = random.sample(range(1, itemnum + 1), sample_nums)
    sample_idx = random_items
    sample_idx_tensor = torch.tensor(sample_idx).to(args.device)
    users = range(1, usernum+1)
    for u in users:
        if len(train[u]) < 1 or len(valid[u]) < 1: continue
        seq = np.zeros([args.maxlen], dtype=np.int32)
        idx = args.maxlen - 1
        for i in reversed(train[u]):
            seq[idx] = i
            # fill the sequence from end to beginning
            idx -= 1
            if idx == -1: break
            # select the max len or all of the training data in the sequence
            # limit the length, seq contains the actual training sequence
        # interacted items
        rated = set(train[u])
        rated.add(0)
        # ground truth item
        ground_truth_idx = [valid[u][0]]
        # collect all indexes, which needs to process on
        process_idx = ground_truth_idx + sample_idx
        predictions = -model.predict(*[np.array(l) for l in [[u], [seq], process_idx]])[0]
        # target distance
        target_d = predictions[0]
        # sampled results
        sample_d = predictions[1:]
        # print(len(sample_d))
        bool_tensor = target_d >= sample_d
        count = torch.sum(bool_tensor).item()
        if count < 10:
            Recall += 1
        sorted_indices = torch.argsort(sample_d)
        sorted_sample_idx = sample_idx_tensor[sorted_indices]
        # take the coverage@10 for all users
        coverage_list+=list(sorted_sample_idx[:10])
        valid_user+=1
    p90_list = [i.item() for i in coverage_list]
    p90_dict = Counter(p90_list)
    p90_sort = sorted(p90_dict.items(), key=lambda x: x[1], reverse=True)
    total_rec = 0
    item_count = 0
    for _, num in p90_sort:
        total_rec+= num
        item_count+= 1
        if total_rec>=0.9*10*usernum:
            break
    return Recall/ valid_user, item_count/sample_nums

In [7]:
r_10, p90_10 = window_eval(model, dataset, args)
r_10, p90_10

(0.6837748344370861, 0.39)

In [324]:
r_10

0.7220198675496688

In [325]:
p90_10[0]

tensor(555, device='cuda:0')

In [4]:
train_user = [7,3,2,8,6,1]
train_user[:-1]

[7, 3, 2, 8, 6]